# 1.) Import the Credit Card Fraud Data From CCLE

In [ ]:
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
drive.mount('/content/gdrive/', force_remount = True)

Mounted at /content/gdrive/


In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/Econ441B/fraudTest.csv")

In [ ]:
df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


# 2.) Select four columns to use as features (one just be trans_date_trans)

In [194]:
df_select=df[['trans_date_trans_time','category','amt','city_pop',"is_fraud"]]

In [195]:
df_select.columns

Index(['trans_date_trans_time', 'category', 'amt', 'city_pop', 'is_fraud'], dtype='object')

# 3.) Create a unique variable out of trans_date.

In [196]:
df_select['trans_date_trans_time']=pd.to_datetime(df_select['trans_date_trans_time'])

<ipython-input-196-9ba00a5dfdf3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['trans_date_trans_time']=pd.to_datetime(df_select['trans_date_trans_time'])


In [197]:
df_select['time_var']=[i.month for i in df_select["trans_date_trans_time"]]

<ipython-input-197-5fcf04a06a3e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['time_var']=[i.month for i in df_select["trans_date_trans_time"]]


In [198]:
X=pd.get_dummies(df_select,['category']).drop(['trans_date_trans_time','is_fraud'],axis=1)
y=df_select['is_fraud']

,amt,city_pop,is_fraud,time_var
count,555719.000000,5.557190e+05,555719.000000,555719.000000
mean,69.392810,8.822189e+04,0.003860,9.508536
std,156.745941,3.003909e+05,0.062008,1.978205
min,1.000000,2.300000e+01,0.000000,6.000000
25%,9.630000,7.410000e+02,0.000000,8.000000
50%,47.290000,2.408000e+03,0.000000,10.000000
75%,83.010000,1.968500e+04,0.000000,12.000000
max,22768.110000,2.906700e+06,1.000000,12.000000


In [199]:
y

0         0
1         0
2         0
3         0
4         0
         ..
555714    0
555715    0
555716    0
555717    0
555718    0
Name: is_fraud, Length: 555719, dtype: int64

# 5.) Train a Logistic regression.

In [200]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_normalized=scaler.fit_transform(X)

In [201]:
from sklearn.linear_model import LogisticRegression

In [202]:
log_reg= LogisticRegression().fit(X_normalized,y)

# 6.) The company you are working for wants to target at a False Positive rate of 5% what threshold should you use? (Use oversampled data)

In [203]:
from sklearn.metrics import roc_curve

In [204]:
pred_probs = log_reg.predict_proba(X_normalized)[:, 1]

In [205]:
fpr, tpr, thresholds = roc_curve(y, pred_probs)

In [206]:
threshold = thresholds[np.argmin(np.abs(fpr - 0.05))]

In [207]:
print('threshold:',threshold)

threshold: 0.010570057941082086


# 7.) If the company makes .02*amt on True transactions and loses -amt on False (Use original data)

In [208]:
df_temp = df_select.copy()

In [209]:
pred1=log_reg.predict_proba(X_normalized)[:,1]
df_temp['pred']=[1 if prob > threshold else 0 for prob in pred1]

In [210]:
df_temp = df_temp[["pred", "is_fraud", "amt"]]

In [211]:
money1=0
money2=0

for i in range(len(df_temp)):
  if df_temp['is_fraud'][i]==0 and df_temp['pred'][i]==0:
    money1=money1+df_temp['amt'][i]
  if df_temp['is_fraud'][i]==1 and df_temp['pred'][i]==0:
    money2=money2+df_temp['amt'][i]
print(0.02*money1-money2)

503902.2153999913


As we can see from the result,the company can  make a profit of $503902

# 8.) Using Logistic Regression Lasso to inform you. Would you use the selected features in a trusted prediction model?

In [212]:
log_reg2=LogisticRegression(penalty='l1',solver="liblinear",max_iter = 10000)
log_reg2.fit(X_normalized,y)

LogisticRegression(max_iter=10000, penalty='l1', solver='liblinear')

In [213]:
log_reg2.coef_

array([[ 0.32943487, -0.12171824, -0.19430607, -0.06807499, -0.0768999 ,
         0.10733589,  0.02470404,  0.43040439, -0.06859387, -0.1124752 ,
        -0.09706374,  0.32538039,  0.        , -0.00954078,  0.40819594,
         0.1052829 , -0.90804273]])

As we can from the coefficients from the table above,only one coefficient of the dummy variable created by the category variable is 0，so we can conclude  that there is no need for us to abandon any four  variables, and my model is a solid model. 